## Testing the API

## Signing in

In [9]:
from atproto import Client

import pandas as pd
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import emoji

import os

In [2]:
cwd = 'C:/Users/dorya/Documents'

In [3]:
data_folder = f'{cwd}/data'

In [4]:
client = Client()

In [36]:
with open(f'{data_folder}/usr_access') as f:
    usrname = f.read().strip()

In [37]:
with open(f'{data_folder}/key_access') as f:
    pwd = f.read().strip()

In [38]:
_ = client.login(usrname, pwd)

## Making a Post

In [39]:
# client.send_post('Hello World')

It worked!

## Building the Framework of the Bot

In [118]:
def create_metadata(row):
    if type(row.redshift.iloc[0]) == str:
        z = row.redshift.iloc[0]
    else:
        z = "%.2f" % ( row.redshift.iloc[0] )
    ra = "%.5f" % row.ra.iloc[0]
    dec = "%.5f" % row.dec.iloc[0]
    clsf = row.galaxy_description.iloc[0]
    survey = row.imaging.iloc[0]
    if "CANDELS-COODS" in survey:
        survey = 'CANDELS-GOODS'
       
    project = row.project.iloc[0]
    if 'Hubble' in project:
        instr = 'Hubble Space Telescope'
    elif 'CANDELS' in project:
        instr = 'Hubble Space Telescope'
    elif 'Galaxy Zoo 2' in project:
        instr = 'Apache Point 2.5m Telescope'

    t_lookback = row.t_lookback.iloc[0]
    if t_lookback < 1:
        tmp = t_lookback * 1000
        t_lookback_string = '%.3f million years' % tmp
    else:
        t_lookback_string = '%.2f billion years' % t_lookback

    random_no = np.random.random()

    if random_no < (1/24):
        
        metadata = (
    """A {}, observed with the {} in the {} survey.
    
It is at redshift {} (lookback time {}) with coordinates ({}, {}).

This classification was made in the {} project.
\U0001f52d
    """.format(
                clsf, instr, survey, z, t_lookback_string, ra, dec, project
            )
        )
    else:
        metadata = (
    """A {}, observed with the {} in the {} survey.
    
It is at redshift {} (lookback time {}) with coordinates ({}, {}).

This classification was made in the {} project.
    """).format(
                clsf, instr, survey, z, t_lookback_string, ra, dec, project
            )

    return metadata

In [41]:
## Functions 
def pull_galaxy_image(url):
    response = requests.get(url)
    img_data = BytesIO(response.content)
    return img_data 

In [52]:
def post(image, metadata, client, clsf, project):

    alt_im_text = 'A {} from the {} project.'.format(clsf, project)

    response = (
        client.send_image(
            text = metadata, 
            image = image, 
            image_alt = 'A Galaxy'
        )
    )
    
    return response

In [43]:
# Selecting a Galaxy to upload.
cat_path = 'https://github.com/vrooje/vrooje.github.io/raw/master/files/gz2_gzh_gzc_gzbot_table.csv'
gal_row = pd.read_csv(cat_path).sample(1)
url = gal_row['image_url'].iloc[0]

In [44]:
gal_row

,subject_id,ra,dec,project,imaging,galaxy_description,redshift,t_lookback,image_url,n_class,...,p_nobar,p_spiral,p_notspiral,p_ring,p_lens,p_disturbed,p_irregular,p_other,p_merger,p_dustlane
11347,AHZ2001sql,150.525248,2.827779,GZ: Hubble,COSMOS,spiral galaxy,0.51,5.281988,http://zoo-hst.s3.amazonaws.com/20190934.jpg,36,...,0.275497,0.677615,0.0,0.031302,0.0,0.031302,0.031302,0.0,0.0,0.0


In [134]:
for i in range(0,24):
    post_string = create_metadata(gal_row)
    if '\U0001f52d' in post_string:
        print(True)
    else:
        print(False)

False
False
False
False
False
True
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False


In [49]:
# Creating the Post
image = pull_galaxy_image(url)
post_string = create_metadata(gal_row)

In [50]:
print(post_string)

A spiral galaxy, observed with the Hubble Space Telescope in the COSMOS survey.
    
It is at redshift 0.51 (lookback time 5.28 billion years) with coordinates (150.52525, 2.82778).

This classification was made in the GZ: Hubble project.
🔭
    


In [53]:
# Posting
response = post(image, post_string, client, gal_row.galaxy_description.iloc[0], gal_row.project.iloc[0])
print(response)

uri='at://did:plc:6itg6aj2yv3gug4ah7ejicsu/app.bsky.feed.post/3kr7bdrkdzf24' cid='bafyreidtdbaowxdriuqnkecz3424gu2mm7fpq6gg3hwg37736wia2ejyli'
